In [1]:
import pandas as pd

ts_data = pd.read_parquet('../data/transformed/ts_data_2022_01.parquet')
# ts_data
ts_one_location = ts_data[ts_data.pickup_location_id == 43]

In [2]:
def get_cutoff_indices(
        data: pd.DataFrame,
        n_features: int,
        step_size: int
) -> list:
    stop_position = len(data) - 1

    subseq_first_idx = 0
    subseq_mid_idx = n_features
    subseq_last_idx = n_features + 1
    indices = []

    while subseq_last_idx <= stop_position:
        indices.append((subseq_first_idx, subseq_mid_idx, subseq_last_idx))

        subseq_first_idx += step_size
        subseq_mid_idx += step_size
        subseq_last_idx += step_size
    
    return indices

In [3]:
n_features = 24
step_size = 1

indices = get_cutoff_indices(
    ts_one_location,
    n_features,
    step_size
)
# indices[:5]

In [4]:
import numpy as np

n_examples = len(indices)
x = np.ndarray(shape=(n_examples, n_features), dtype=np.float32)
y =  np.ndarray(shape=(n_examples), dtype=np.float32)

pickup_hours = []

for i, idx in enumerate(indices):
    x[i, :] = ts_one_location.iloc[idx[0]:idx[1]]['rides'].values
    y[i] = ts_one_location.iloc[idx[1]]['rides']
    pickup_hours.append(ts_one_location.iloc[idx[1]]['pickup_hour'])

In [5]:
features_one_location = pd.DataFrame(
    x,
    columns= [f"rides_previous_{i+1}_hour" for i in reversed(range(n_features))]
)
features_one_location

,rides_previous_24_hour,rides_previous_23_hour,rides_previous_22_hour,rides_previous_21_hour,rides_previous_20_hour,rides_previous_19_hour,rides_previous_18_hour,rides_previous_17_hour,rides_previous_16_hour,rides_previous_15_hour,...,rides_previous_10_hour,rides_previous_9_hour,rides_previous_8_hour,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour
0,97.0,60.0,22.0,8.0,6.0,5.0,3.0,10.0,7.0,19.0,...,70.0,94.0,87.0,73.0,34.0,32.0,22.0,16.0,18.0,6.0
1,60.0,22.0,8.0,6.0,5.0,3.0,10.0,7.0,19.0,24.0,...,94.0,87.0,73.0,34.0,32.0,22.0,16.0,18.0,6.0,3.0
2,22.0,8.0,6.0,5.0,3.0,10.0,7.0,19.0,24.0,39.0,...,87.0,73.0,34.0,32.0,22.0,16.0,18.0,6.0,3.0,1.0
3,8.0,6.0,5.0,3.0,10.0,7.0,19.0,24.0,39.0,35.0,...,73.0,34.0,32.0,22.0,16.0,18.0,6.0,3.0,1.0,1.0
4,6.0,5.0,3.0,10.0,7.0,19.0,24.0,39.0,35.0,77.0,...,34.0,32.0,22.0,16.0,18.0,6.0,3.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
714,52.0,36.0,28.0,16.0,13.0,8.0,1.0,1.0,2.0,1.0,...,78.0,74.0,66.0,91.0,117.0,100.0,106.0,147.0,121.0,102.0
715,36.0,28.0,16.0,13.0,8.0,1.0,1.0,2.0,1.0,1.0,...,74.0,66.0,91.0,117.0,100.0,106.0,147.0,121.0,102.0,66.0
716,28.0,16.0,13.0,8.0,1.0,1.0,2.0,1.0,1.0,4.0,...,66.0,91.0,117.0,100.0,106.0,147.0,121.0,102.0,66.0,61.0
717,16.0,13.0,8.0,1.0,1.0,2.0,1.0,1.0,4.0,9.0,...,91.0,117.0,100.0,106.0,147.0,121.0,102.0,66.0,61.0,73.0


In [6]:
targets_one_location = pd.DataFrame(y, columns=["target_rides_next-hour"])

In [21]:
from tqdm import tqdm

def transform_ts_data_into_features_and_target(
        ts_data: pd.DataFrame,
        input_seq_len: int,
        step_size: int,
)-> pd.DataFrame:
    """
    Slices and transposes data from time-series format into a (features, target)
    format that we can use to train Supervised ML Models
    """

    assert set(ts_data.columns) == {'pickup_hour', 'rides', 'pickup_location_id'}

    location_ids = ts_data['pickup_location_id'].unique()
    features = pd.DataFrame()
    targets = pd.DataFrame()

    for location_id in tqdm(location_ids):

        ts_data_one_location = ts_data.loc[
            ts_data.pickup_location_id == location_id,
            ['pickup_hour', 'rides']
        ]


        indices = get_cutoff_indices(
            ts_data_one_location,
            input_seq_len,
            step_size
        )

        n_examples = len(indices)
        x = np.ndarray(shape=(n_examples, input_seq_len), dtype=np.float32)
        y =  np.ndarray(shape=(n_examples), dtype=np.float32)

        pickup_hours = []

        for i, idx in enumerate(indices):
            x[i, :] = ts_one_location.iloc[idx[0]:idx[1]]['rides'].values
            y[i] = ts_one_location.iloc[idx[1]]['rides']
            pickup_hours.append(ts_one_location.iloc[idx[1]]['pickup_hour'])

        features_one_location = pd.DataFrame(
            x,
            columns= [f"rides_previous_{i+1}_hour" for i in reversed(range(input_seq_len))]
        )
        features_one_location['pickup_hours'] = pickup_hours
        features_one_location['pickup_location_id'] = location_id

        targets_one_location = pd.DataFrame(y, columns=["target_rides_next_hour"])

        features = pd.concat([features, features_one_location])
        targets = pd.concat([targets, targets_one_location])

    features.reset_index(inplace=True, drop=True)
    targets.reset_index(inplace=True, drop=True)

    return features, targets['target_rides_next_hour']

In [22]:
features, targets = transform_ts_data_into_features_and_target(
    ts_data,
    input_seq_len=24*7,
    step_size=24
)

print(f'{features.shape}')
print(f'{targets.shape}')

100%|██████████| 257/257 [00:02<00:00, 119.98it/s]

(6168, 170)
(6168,)
